## Истражите K-Means кластерисање користећи R и принципе уређених података.

### [**Квиз пре предавања**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

У овом часу ћете научити како да креирате кластере користећи пакет Tidymodels и друге пакете из R екосистема (назваћемо их пријатељи 🧑‍🤝‍🧑), као и нигеријски музички скуп података који сте раније увезли. Покрићемо основе K-Means кластерисања. Имајте на уму да, као што сте научили у претходном часу, постоји много начина за рад са кластерима, а метод који користите зависи од ваших података. Пробаћемо K-Means јер је то најчешћа техника кластерисања. Хајде да почнемо!

Термини које ћете научити:

-   Силуетно оцењивање

-   Метода лакта

-   Инерција

-   Варијанса

### **Увод**

[K-Means кластерисање](https://wikipedia.org/wiki/K-means_clustering) је метода која потиче из области обраде сигнала. Користи се за поделу и груписање података у `k кластера` на основу сличности њихових карактеристика.

Кластери се могу визуализовати као [Воронојеви дијаграми](https://wikipedia.org/wiki/Voronoi_diagram), који укључују тачку (или 'семе') и њен одговарајући регион.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Инфографика од Џен Лупер</figcaption>


K-Means кластерисање има следеће кораке:

1.  Научник за податке започиње одређивањем жељеног броја кластера који ће бити креирани.

2.  Затим алгоритам насумично бира K опсервација из скупа података које ће служити као почетни центри за кластере (тј. центроиде).

3.  Затим се свака од преосталих опсервација додељује најближем центроиду.

4.  Затим се рачуна нова средња вредност сваког кластера и центроид се помера на ту средњу вредност.

5.  Сада када су центри поново израчунати, свака опсервација се поново проверава да ли би могла бити ближа другом кластеру. Сви објекти се поново додељују користећи ажуриране средње вредности кластера. Кораци доделе кластера и ажурирања центроида се итеративно понављају док се доделе кластера не престану мењати (тј. када се постигне конвергенција). Типично, алгоритам се завршава када свака нова итерација резултира занемарљивим померањем центроида и кластери постану статични.

<div>

> Имајте на уму да због насумичности почетних K опсервација које се користе као почетни центроиди, можемо добити благо различите резултате сваки пут када применимо процедуру. Из тог разлога, већина алгоритама користи неколико *насумичних почетака* и бира итерацију са најнижом WCSS. Због тога се снажно препоручује да увек покрећете K-Means са неколико вредности *nstart* како бисте избегли *непожељан локални оптимум.*

</div>

Ова кратка анимација користећи [уметничке радове](https://github.com/allisonhorst/stats-illustrations) Алисон Хорст објашњава процес кластерисања:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Уметнички рад од @allison_horst</figcaption>



Основно питање које се јавља у кластерисању је следеће: како знате на колико кластера да поделите своје податке? Један недостатак коришћења K-Means укључује чињеницу да ћете морати да одредите `k`, односно број `центроида`. Срећом, `метода лакта` помаже да се процени добра почетна вредност за `k`. Ускоро ћете је испробати.

### 

**Предуслов**

Наставићемо тачно тамо где смо стали у [претходном часу](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), где смо анализирали скуп података, направили много визуализација и филтрирали скуп података на опсервације од интереса. Обавезно погледајте!

Биће нам потребни неки пакети да завршимо овај модул. Можете их инсталирати као: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Алтернативно, скрипта испод проверава да ли имате пакете потребне за завршетак овог модула и инсталира их за вас у случају да неки недостају.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Хајде да кренемо одмах!

## 1. Плес са подацима: Сузимо избор на 3 најпопуларнија музичка жанра

Ово је преглед онога што смо радили у претходној лекцији. Хајде да мало анализирамо податке!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 То је прошло добро!

## 2. Више истраживања података.

Колико су ови подаци чисти? Хајде да проверимо постојање екстремних вредности користећи box plot графиконе. Фокусираћемо се на нумеричке колоне са мање екстремних вредности (иако можете уклонити екстремне вредности). Box plot графикони могу показати опсег података и помоћи у избору колона које ћемо користити. Напомена: Box plot графикони не приказују варијансу, што је важан елемент за добро кластерисане податке. Молимо вас да погледате [ову дискусију](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) за више информација.

[Box plot графикони](https://en.wikipedia.org/wiki/Box_plot) се користе за графичко приказивање расподеле `нумеричких` података, па хајде да почнемо са *избором* свих нумеричких колона заједно са популарним музичким жанровима.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Погледајте како селектор `where` олакшава овај процес 💁? Истражите и друге сличне функције [овде](https://tidyselect.r-lib.org/).

Пошто ћемо правити boxplot за сваку нумеричку карактеристику и желимо да избегнемо коришћење петљи, хајде да преобликујемо наше податке у *дужи* формат који ће нам омогућити да искористимо `facets` - подграфиконе који приказују појединачне подскупове података.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Сада много дужи! Време је за мало `ggplots`! Па који `geom` ћемо користити?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Лако-gg!

Сада можемо видети да су подаци мало "бучни": посматрајући сваку колону као boxplot, можете уочити екстремне вредности. Могли бисте проћи кроз скуп података и уклонити те екстремне вредности, али то би учинило податке прилично минималним.

За сада, хајде да изаберемо које ћемо колоне користити за наш кластеринг задатак. Изабраћемо нумеричке колоне са сличним опсезима. Могли бисмо да енкодирамо `artist_top_genre` као нумеричку вредност, али ћемо је за сада изоставити.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Израчунавање k-means кластеризације у R-у

Можемо израчунати k-means кластеризацију у R-у помоћу уграђене функције `kmeans`, погледајте `help("kmeans()")`. Функција `kmeans()` прихвата податке у облику табеле са свим нумеричким колонама као свој примарни аргумент.

Први корак при коришћењу k-means кластеризације је да се одреди број кластера (k) који ће бити генерисани у коначном решењу. Знамо да постоје 3 жанра песама које смо издвојили из скупа података, па хајде да пробамо са 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Објекат kmeans садржи неколико информација које су добро објашњене у `help("kmeans()")`. За сада, хајде да се фокусирамо на неколико њих. Видимо да су подаци груписани у 3 кластера величина 65, 110, 111. Излаз такође садржи центаре кластера (средине) за 3 групе кроз 5 променљивих.

Вектор кластерисања представља доделу кластера за сваку опсервацију. Хајде да користимо функцију `augment` како бисмо додали доделу кластера оригиналном скупу података.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Одлично, управо смо поделили наш скуп података у 3 групе. Па, колико је добро наше кластерисање 🤷? Хајде да погледамо `Silhouette score`.

### **Silhouette score**

[Silhouette анализа](https://en.wikipedia.org/wiki/Silhouette_(clustering)) може се користити за проучавање удаљености раздвајања између добијених кластера. Овај скор варира од -1 до 1, и ако је скор близу 1, кластер је густ и добро одвојен од других кластера. Вредност близу 0 представља преклапајуће кластере са узорцима који су веома близу граници одлуке суседних кластера. [извор](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Метод просечног silhouette-а израчунава просечан silhouette за различите вредности *k*. Висок просечан silhouette скор указује на добро кластерисање.

Функција `silhouette` у пакету за кластерисање користи се за израчунавање просечне ширине silhouette-а.

> Silhouette се може израчунати помоћу било које [метрике удаљености](https://en.wikipedia.org/wiki/Distance "Distance"), као што су [Еуклидска удаљеност](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") или [Манхатен удаљеност](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") коју смо разматрали у [претходном часу](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Наш резултат је **0.549**, што је тачно на средини. Ово указује да наши подаци нису нарочито погодни за ову врсту кластерисања. Хајде да видимо да ли можемо визуелно да потврдимо ову претпоставку. [factoextra пакет](https://rpkgs.datanovia.com/factoextra/index.html) пружа функције (`fviz_cluster()`) за визуелизацију кластерисања.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Преклапање у кластерима указује на то да наши подаци нису нарочито погодни за ову врсту кластерисања, али хајде да наставимо.

## 4. Одређивање оптималног броја кластера

Основно питање које се често јавља код K-Means кластерисања је следеће - без познатих ознака класа, како знати на колико кластера треба поделити податке?

Један од начина да то сазнамо је да користимо узорак података за `креирање серије модела кластерисања` са повећавајућим бројем кластера (нпр. од 1 до 10), и да проценимо метрике кластерисања као што је **Silhouette score.**

Хајде да одредимо оптималан број кластера тако што ћемо израчунати алгоритам кластерисања за различите вредности *k* и проценити **Within Cluster Sum of Squares** (WCSS). Укупна сума квадрата унутар кластера (WCSS) мери компактност кластерисања, и желимо да она буде што мања, јер ниже вредности значе да су тачке података ближе једна другој.

Хајде да истражимо ефекат различитих избора `k`, од 1 до 10, на ово кластерисање.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Сада када имамо укупну суму квадрата унутар кластера (tot.withinss) за сваки алгоритам кластерисања са центром *k*, користимо [метод лакта](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) да бисмо пронашли оптималан број кластера. Овај метод се састоји од графичког приказивања WCSS као функције броја кластера и одабирања [лакта криве](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") као броја кластера који ће се користити.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


График приказује значајно смањење WCSS (што значи већу *збијеност*) како се број кластера повећава са један на два, и још једно приметно смањење са два на три кластера. Након тога, смањење је мање изражено, што резултира `елбоу` 💪 на графику око три кластера. Ово је добар показатељ да постоје два до три разумно добро одвојена кластера тачака података.

Сада можемо наставити и издвојити модел кластерисања где је `k = 3`:

> `pull()`: користи се за издвајање једне колоне
>
> `pluck()`: користи се за индексирање структура података као што су листе


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Одлично! Хајде да визуализујемо добијене кластере. Желите ли мало интерактивности уз помоћ `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Можда бисмо очекивали да сваки кластер (представљен различитим бојама) има јасно одвојене жанрове (представљене различитим облицима).

Хајде да погледамо тачност модела.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Tačnost ovog modela nije loša, ali nije ni sjajna. Moguće je da podaci nisu pogodni za K-Means klasterizaciju. Ovi podaci su previše neuravnoteženi, slabo povezani i postoji prevelika varijansa između vrednosti kolona da bi se dobro grupisali. Zapravo, klasteri koji se formiraju verovatno su značajno pod uticajem ili iskrivljeni zbog tri kategorije žanrova koje smo gore definisali.

Ipak, ovo je bio prilično poučan proces!

U dokumentaciji Scikit-learn-a možete videti da model poput ovog, sa klasterima koji nisu dobro definisani, ima problem sa 'varijansom':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografika iz Scikit-learn-a</figcaption>



## **Varijansa**

Varijansa se definiše kao "prosek kvadrata razlika od srednje vrednosti" [izvor](https://www.mathsisfun.com/data/standard-deviation.html). U kontekstu ovog problema klasterizacije, odnosi se na podatke kod kojih vrednosti našeg skupa podataka imaju tendenciju da se previše udalje od srednje vrednosti.

✅ Ovo je odličan trenutak da razmislite o svim načinima na koje biste mogli da rešite ovaj problem. Da li da malo prilagodite podatke? Koristite različite kolone? Primenite drugačiji algoritam? Savet: Probajte [skaliranje podataka](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) kako biste ih normalizovali i testirali druge kolone.

> Probajte ovaj '[kalkulator varijanse](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' da biste bolje razumeli koncept.

------------------------------------------------------------------------

## **🚀Izazov**

Provedite neko vreme sa ovim beležnicom, podešavajući parametre. Možete li poboljšati tačnost modela čišćenjem podataka (na primer, uklanjanjem ekstremnih vrednosti)? Možete koristiti težine kako biste određenim uzorcima podataka dali veću važnost. Šta još možete učiniti da kreirate bolje klastere?

Savet: Probajte da skalirate podatke. U beležnici postoji komentarisani kod koji dodaje standardno skaliranje kako bi kolone podataka bile sličnije u smislu opsega. Videćete da, iako se silueta skora smanjuje, 'prelom' u grafiku lakta postaje glađi. To je zato što ostavljanje podataka neskaliranih omogućava podacima sa manjom varijansom da imaju veću težinu. Pročitajte više o ovom problemu [ovde](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Kviz nakon predavanja**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Pregled i samostalno učenje**

-   Pogledajte simulator za K-Means [kao što je ovaj](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Možete koristiti ovaj alat za vizualizaciju uzoraka podataka i određivanje njihovih centara. Možete uređivati nasumičnost podataka, broj klastera i broj centara. Da li vam ovo pomaže da steknete ideju o tome kako se podaci mogu grupisati?

-   Takođe, pogledajte [ovaj materijal o K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) sa Stanforda.

Želite da testirate svoje novo stečene veštine klasterizacije na skupovima podataka koji su pogodni za K-Means klasterizaciju? Pogledajte:

-   [Treniranje i evaluacija modela klasterizacije](https://rpubs.com/eR_ic/clustering) koristeći Tidymodels i slične alate

-   [K-Means analiza klastera](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-Means klasterizacija sa principima urednih podataka](https://www.tidymodels.org/learn/statistics/k-means/)

## **Zadatak**

[Probajte različite metode klasterizacije](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## HVALA:

[Jen Looper](https://www.twitter.com/jenlooper) za kreiranje originalne Python verzije ovog modula ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) za kreiranje neverovatnih ilustracija koje čine R pristupačnijim i zanimljivijim. Pronađite više ilustracija u njenoj [galeriji](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Srećno učenje,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Ilustracija @allison_horst</figcaption>



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.
